In [1]:
!pwd

/Users/vascomeerman/Coding/Harvard/CS215/Project/realworld-stylegan2-encoder


In [2]:
import os
import sys

# Path to the psp repo in our shared folder (I believe)
psp_path = "v3_psp"

# Append the directory to our python path
sys.path.append(psp_path)
# Set current working directory
os.chdir(psp_path)

In [3]:
# !ls

In [4]:
import time
import pprint
from argparse import Namespace
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

# sys.path.append(".")
# sys.path.append("..")


# from datasets import augmentations
from utils.common import tensor2im, log_input_image
from models.psp import pSp

In [5]:
torch.__version__

'1.10.0'

In [6]:
# # I am not using a GPU here, if you are, move it to cuda
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu" # UNCOMMENT FOR CUDA 
print(f"Running on device -> {device}")

Running on device -> cpu


In [7]:
experiment_type = 'ffhq_encode'

In [8]:
EXPERIMENT_DATA_ARGS = {
    "ffhq_encode": {
        "model_path": "pretrained_models/psp_ffhq_encode.pt",
        "image_path": "notebooks/images/input_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    },
}
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[experiment_type]

## Analysis of opset versions

Versions that DON'T work (cpu)
- 9
- 10
- 13
- 14


Version that do work withouth attenfallback (cpu)
- 11
- 12


Versions only work with attentfallback (cpu):
- 



In [25]:
model_path = EXPERIMENT_ARGS['model_path']
OPSET_VERSION = 9

In [1]:
# Output of shape 1000, confidence scores for each of the imagenet classes
# Now we will save this model.
import torch.onnx

# # I am not using a GPU here, if you are, move it to cuda
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

def export_onnx_model(model, onnx_model_path):
    with torch.no_grad():
        # Bring full model to cpu for conversion
        # model.to(device)

        # Format input image into a batch format the model can use
        # inp_batch = transformed_image1.unsqueeze(0)
        inp_batch = torch.randn(1, 3, 256, 256).to(device)


        # Run the model once on the same input we give it for onnx to run
        # _, _ = model(img_tensor.float(), randomize_noise=False, return_latents= True)
        print("\n\n --------------ONNXX LETS GO ------------------\n\n")

        # Inputs needed for onnx to run 1 inference session so it can determine the graph it needs to convert
        inputs = (inp_batch.float(),
                  {"randomize_noise": False,
                  "return_latents": True})
        
        # Convert our pytroch model to onnx, so we can later on convert it to tf
        torch.onnx.export(model,                                            
                          inputs,
                          onnx_model_path,                                  
                          opset_version=OPSET_VERSION,
                          do_constant_folding=True,
                          # See: https://github.com/pytorch/fairseq/issues/3395
                          operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                          export_params=True,
                        #   input_names=['input'],
                        #   output_names=['output'],               
                          # input_names=['input_ids',                         
                          #              'input_code',
                          #              'return_latents']
                          )
        
        print("ONNX Model exported to {0}".format(onnx_model_path))



def export_new_model():
    ckpt2 = torch.load(model_path, map_location='cpu')

    # update the training options
    opts2 = ckpt2['opts']
    if device == "cpu":
        opts2['device'] = "cpu"

    opts2['checkpoint_path'] = model_path
    if 'learn_in_w' not in opts2:
        opts2['learn_in_w'] = False
    if 'output_size' not in opts2:
        opts2['output_size'] = 1024


    opts2 = Namespace(**opts2)
    new_net = pSp(opts2)
    new_net.eval()

    if device != "cpu":
        new_net.cuda() # Don't move model to cuda for conversion
    else:
        print(f"No cuda -> Running on {device}")

    export_onnx_model(new_net, f"pretrained_models/psp_clean_ops{OPSET_VERSION}.onnx")

    
# export_new_model()

## Loading the ONNX model to TF

In [ ]:
CURRENT_MODEL_CPU = "psp_clean_"
WEIGHT_PATH = 'pretrained_models/'

In [2]:
import onnx
from onnx_tf.backend import prepare
tic = time.time()

# Load ONNX model and convert to TensorFlow format
model_psp_onnx = onnx.load(f'{CURRENT_MODEL_CPU}{CURRENT_MODEL_CPU}.onnx')
# model_decoder_onnx = onnx.load('../output/decoder.onnx')

# Load onnx model into onnx_tf representation
tf_psp_rep = prepare(model_psp_onnx, device="CPU")
# tf_decoder_rep = prepare(model_decoder_onnx)

# Timer ends
toc = time.time()
print('Loading onnx models to tf, took {:.4f} seconds.'.format(toc - tic))